In [31]:
!pip install vncorenlp


[notice] A new release of pip available: 22.3 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [32]:
from vncorenlp import VnCoreNLP

rdrsegmenter = VnCoreNLP(
    r"D:\Download\VnCoreNLP-1.1.1\VnCoreNLP-1.1.1\VnCoreNLP-1.1.1.jar",
    annotators="wseg",
    max_heap_size='-Xmx2g'
)
print(rdrsegmenter.tokenize("Hồ Chí Minh là lãnh tụ vĩ đại"))

[['Hồ_Chí_Minh', 'là', 'lãnh_tụ', 'vĩ_đại']]


In [33]:
input_file = r"D:\NCKH\estimate_question_difficulty\Embedding\Input\qa_processed.txt"
output_file = r"D:\NCKH\estimate_question_difficulty\Embedding\Output_ws\qa_processed_ws.txt"

with open(input_file, "r", encoding="utf-8") as f:
    lines = f.readlines()

with open(output_file, "w", encoding="utf-8") as f:
    for line in lines:
        line = line.strip()
        if line:
            segmented = rdrsegmenter.tokenize(line)
            f.write(" ".join(segmented[0]) + "\n")

In [35]:
import json
import re

# Đọc file txt
with open(r"D:\NCKH\estimate_question_difficulty\Embedding\Output_ws\qa_processed_ws.txt", "r", encoding="utf-8") as f:
    text = f.read()

# Tách các block câu hỏi bằng chuỗi gạch ngang dài
blocks = [b.strip() for b in text.split("- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -") if b.strip()]

questions = []
for i, block in enumerate(blocks, start=1):
    subject_match = re.search(r"Môn\s*:\s*(.+)", block)
    subject = subject_match.group(1).strip() if subject_match else None

    question_match = re.search(r"Câu hỏi\s*:\s*(.+)", block)
    question = question_match.group(1).strip() if question_match else None

    options = re.findall(r"\d+\s*\.\s*([A-D])\s+(.+)", block)
    options_dict = {opt[0]: opt[1].strip() for opt in options} if options else {}

    answer_match = re.search(r"Đáp[_ ]?án đúng\s*:\s*([A-D](?:[,;]\s*[A-D])*)", block)
    if answer_match:
        answer_keys = [a.strip() for a in re.split(r"[,;]", answer_match.group(1))]
    else:
        answer_keys = []

    if not answer_keys:
        continue
    valid_answers = [options_dict[a] for a in answer_keys if a in options_dict]

    if subject and question and options_dict and all(a in options_dict for a in answer_keys):
        options_list = list(options_dict.values())

        questions.append({
            "id": i,
            "subject": subject,
            "question": question,
            "options": options_list,
            "answer": valid_answers if len(valid_answers) > 1 else valid_answers[0]
        })
    else:
        print(f"Bỏ qua câu {i}, đáp án không khớp với options.")

with open(r"D:\NCKH\estimate_question_difficulty\Embedding\Output_ws\questions.json", "w", encoding="utf-8") as f:
    json.dump(questions, f, ensure_ascii=False, indent=4)

print(f"✅ Đã chuyển thành công {len(questions)} câu hỏi sang questions.json")

Bỏ qua câu 40, đáp án không khớp với options.
✅ Đã chuyển thành công 59 câu hỏi sang questions.json
